<h1 style="text-align:center">ASAT </h1>

<h2 style="text-align:center;margin-bottom:50px">Aeronautics Recruitment: Software Development
Subsystem
</h2>


<div style="margin-bottom: 100px">
    <h3>Autopilot Overview</h3>
    <p>For a UAV to be able to function autonomously, an interactive understanding of its environment is essential. To achieve that, the vehicle needs to integrate a series of sensors that will provide information on its location, orientation, speed, altitude, etc. Such sensors are a 3-axis gyroscope, a GPS (Global Positioning System), an accelerometer, an altimeter, and an airspeed sensor. Due to the objective of project Phoenix (fire detection), a thermal and a regular camera are required. For the UAV to be able to communicate and detect other aircraft, it can use ADS-B technology (Automatic Dependent Surveillance-Broadcast). The hardware mentioned above should comply with the pixhawk open standards to avoid unnecessary incompatibilities</p>
    <p>Apart from the sensor's data, the autopilot needs additional input from the user to plan its route accordingly. The user shall specify the destination, flight altitude, and speed of the aircraft. There should also be an option to add waypoints. With the purpose of fire detection, it should include a mode that allows the user to select an area for scanning, letting the UAV roam above it.</p>
    <p>To process this data, we are to use open source autopilot software, such as Px4 and ArduPilot. Both of these software use MAVlink (Micro Air Vehicle communication protocol) to communicate with the user/control station. To set the input parameters (route, waypoints) and monitor the vehicle, we can use a MAVlink based control station, like QGroundControl.</p>
    <p> In case the user chooses an impossible route, an error should appear asking them to select different parameters or manually operate the UAV. If the error occurs while on air (obstacle, weather conditions, etc) the autopilot should re-plan its path using a path optimizing algorithm, or -in extreme conditions- land to the nearest safe point. Safe points could be pre-determined by the user for extra security.</p>
    <p>Finally, to ensure the correct operation of the system, we can run simulations in software such as AirSim (SITL), Gazebo, jMAVSim.</p>
</div>

<div>
    <h3>Dijkstra's Algorithm</h3>
    <p>Dijkstra's Algorithm is an algorithm for finding the shortest paths between nodes in a graph. In this implementation, we will be using a node as our base node (source) and computing the shortest path from the source to every other node.</p>
</div>

<p>To use certain math operations we import NumPy </p>

In [80]:
import numpy as np

<p>We are given 5 nodes:<br>
Node 1 = connection[0] = [0,1,0,3,10]<br>
Node 2 = connction[1] = [1, 0, 5, 0, 0]<br>
.<br>.<br>.<br>
Where each number inside the lists represents the weight (or difficulty) of the connection between the nodes.<br>
For exampe: node 1 is not connected to itself (0), is connected to node 2 with a weight of 1 (easy to reach), is not connected to node 3, is connected to node 4 with a weight of 3 (slightly more difficult to reach), and is connected to node 5 with a weight of 10 (difficult to reach). <br>
Based on this data we create the following list (you can think of it as an array, though in reality it is a python list):</p>

In [81]:
connections = [
    [0, 1, 0, 3, 10],
    [1, 0, 5, 0, 0],
    [0, 5, 0, 2, 1],
    [3, 0, 2, 0, 6],
    [10, 0, 1, 6, 0]
    ]

<p>We know that our source node is node 1, so we use its index [0] to specify its value :</p>

In [82]:
baseNode=0

<p>
    We will be tracking the total path weight of each node (from the source node) in a list named weight_sum wich will contain as many elements as the nodes (5).<br>
    We will also create a list named unvisited which will contain the indexes (of the weight_sum list) of the nodes that have not yet been completed.,<br>
    The minimum could be any positive number, we just want to initialize it.<br>
    Finally, we set the weight of the basenode to 0 (we are already there so the path is 0)
</p>

In [83]:
weight_sum=[]
unvisited=[]
minimum=0
for i in range(0, len(connections)):
    weight_sum.append(np.infty)
    unvisited.append(i)
weight_sum[baseNode]=0

<p>The following piece of code finds the minimum weight_sum out of all the unvisited indexes, and visits that node to examine its neighboring nodes. <br>If the weight of the connection is 0, it ignores it (they are not connected). For non zero values it checks if this new path is shorter than the already registered and it updates the value to the minimum.<br> It removes the visited node from the unvisited list and resets the minimum to a large enough value that will allow the correct function of the loop.</p>

In [84]:
while len(unvisited) != 0:
    for i in unvisited:
        if minimum>=weight_sum[i]:
            minimum=weight_sum[i]
    i=weight_sum.index(minimum)
    unvisited.remove(i)
    for j in range(0, len(connections)):
        if connections[i][j]!=0:
            weight_sum[j]=min(weight_sum[j], weight_sum[i]+connections[i][j])
    minimum=max(weight_sum)

<p>The only thing left is to print the minimum weight sums for each node to the console:</p>

In [93]:
print("==========================================")
print("Starting from node " + str(baseNode+1) + " :")
print("Minimum distance to other nodes:\n")
print("Node \t\t Weight-Sum/Distance\n------------------------------------------")

for j in range(0, len(connections)):
    print(j+1, "\t\t", weight_sum[j])

print("\n==========================================")

Starting from node 1 :
Minimum distance to other nodes:

Node 		 Weight-Sum/Distance
------------------------------------------
1 		 0
2 		 1
3 		 5
4 		 3
5 		 6



<h3>
    Digit Classification
</h3>

In [94]:
import numpy as np
import tensorflow as tf

In [95]:
mnist=tf.keras.datasets.mnist

In [96]:
(training_data,training_labels),(test_data,test_labels)=mnist.load_data()

In [97]:
training_data=training_data/255
test_data=test_data/255

In [98]:
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(120,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

In [99]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

In [101]:
model.fit(training_data,training_labels,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0355 - accuracy: 0.9887
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0283 - accuracy: 0.9911
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0220 - accuracy: 0.9930
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0181 - accuracy: 0.9947
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0160 - accuracy: 0.9951


In [102]:
model.evaluate(test_data,test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0884 - accuracy: 0.9775


[0.08837443590164185, 0.9775000214576721]

In [103]:
predictions=model.predict(test_data)
np.set_printoptions(suppress=True)
print(test_labels[0])
print(predictions[0])

7
[0.00000001 0.         0.00000001 0.00000004 0.         0.
 0.         1.         0.         0.00000002]
